# Import project path

In [2]:
import sys
sys.path.append("D:/git/NewUserPredict")
from tools.config_file import NewUserPredictParams
params = NewUserPredictParams()

# Import other packages

In [4]:
import pandas as pd

In [106]:
df_train = pd.read_csv(params.train_csv)
df_test = pd.read_csv(params.test_csv)

In [107]:
df_train

,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,x7,x8,target
0,0,26,"{""key3"":""67804"",""key2"":""650""}",1689673468244,4,0,41,107,206,1,0,1,0
1,1,26,"{""key3"":""67804"",""key2"":""484""}",1689082941469,4,0,41,24,283,4,8,1,0
2,2,8,unknown,1689407393040,4,0,41,71,288,4,7,1,0
3,3,11,unknown,1689467815688,1,3,41,17,366,1,6,1,0
4,4,26,"{""key3"":""67804"",""key2"":""650""}",1689491751442,0,3,41,92,383,4,8,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
620351,620351,2,"{""key4"":""14"",""key5"":""14""}",1688990850527,4,0,41,104,37,4,4,1,0
620352,620352,8,unknown,1689490302305,4,0,41,104,37,1,6,1,0
620353,620353,41,"{""key1"":""274"",""key2"":""446""}",1688892078584,0,2,41,104,37,4,2,0,1
620354,620354,26,"{""key3"":""67804"",""key2"":""650""}",1689652960171,4,0,41,17,366,1,6,1,0


In [108]:
df_train['common_ts_dt'] = pd.to_datetime(df_train['common_ts'], unit='ms')
df_train['year'] = df_train['common_ts_dt'].dt.year
df_train['month'] = df_train['common_ts_dt'].dt.month
df_train['day'] = df_train['common_ts_dt'].dt.day
df_train['dayofweek'] = df_train['common_ts_dt'].dt.dayofweek + 1
df_train['hour'] = df_train['common_ts_dt'].dt.hour

In [110]:
aggs = {
    'target': ['mean', 'skew']
}
df_grouped_hour = df_train.groupby(['hour']).agg(aggs).reset_index()
df_grouped_hour.columns = ['hour', 'hour_target_mean', 'hour_target_skew']
df_train = pd.merge(df_train, df_grouped_hour, how='left', on='hour')
df_train

,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,...,x8,target,common_ts_dt,year,month,day,dayofweek,hour,hour_target_mean,hour_target_skew
0,0,26,"{""key3"":""67804"",""key2"":""650""}",1689673468244,4,0,41,107,206,1,...,1,0,2023-07-18 09:44:28.244,2023,7,18,2,9,0.161633,1.838453
1,1,26,"{""key3"":""67804"",""key2"":""484""}",1689082941469,4,0,41,24,283,4,...,1,0,2023-07-11 13:42:21.469,2023,7,11,2,13,0.154592,1.910965
2,2,8,unknown,1689407393040,4,0,41,71,288,4,...,1,0,2023-07-15 07:49:53.040,2023,7,15,6,7,0.179153,1.673404
3,3,11,unknown,1689467815688,1,3,41,17,366,1,...,1,0,2023-07-16 00:36:55.688,2023,7,16,7,0,0.074630,3.237474
4,4,26,"{""key3"":""67804"",""key2"":""650""}",1689491751442,0,3,41,92,383,4,...,1,0,2023-07-16 07:15:51.442,2023,7,16,7,7,0.179153,1.673404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620351,620351,2,"{""key4"":""14"",""key5"":""14""}",1688990850527,4,0,41,104,37,4,...,1,0,2023-07-10 12:07:30.527,2023,7,10,1,12,0.145397,2.012027
620352,620352,8,unknown,1689490302305,4,0,41,104,37,1,...,1,0,2023-07-16 06:51:42.305,2023,7,16,7,6,0.139946,2.075754
620353,620353,41,"{""key1"":""274"",""key2"":""446""}",1688892078584,0,2,41,104,37,4,...,0,1,2023-07-09 08:41:18.584,2023,7,9,7,8,0.163471,1.820149
620354,620354,26,"{""key3"":""67804"",""key2"":""650""}",1689652960171,4,0,41,17,366,1,...,1,0,2023-07-18 04:02:40.171,2023,7,18,2,4,0.140387,2.070486


In [111]:
df_train['hour_section'] = ((df_train['hour'] ) // 6).astype('int64')
df_grouped_hour_section = df_train.groupby('hour_section').agg(aggs).reset_index()
df_grouped_hour_section.columns = ['hour_section', 'hour_section_target_mean', 'hour_section_target_skew']
df_train = pd.merge(df_train, df_grouped_hour_section, how='left', on='hour_section')
df_train

,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,...,year,month,day,dayofweek,hour,hour_target_mean,hour_target_skew,hour_section,hour_section_target_mean,hour_section_target_skew
0,0,26,"{""key3"":""67804"",""key2"":""650""}",1689673468244,4,0,41,107,206,1,...,2023,7,18,2,9,0.161633,1.838453,1,0.160579,1.849007
1,1,26,"{""key3"":""67804"",""key2"":""484""}",1689082941469,4,0,41,24,283,4,...,2023,7,11,2,13,0.154592,1.910965,2,0.148067,1.981816
2,2,8,unknown,1689407393040,4,0,41,71,288,4,...,2023,7,15,6,7,0.179153,1.673404,1,0.160579,1.849007
3,3,11,unknown,1689467815688,1,3,41,17,366,1,...,2023,7,16,7,0,0.074630,3.237474,0,0.126957,2.241021
4,4,26,"{""key3"":""67804"",""key2"":""650""}",1689491751442,0,3,41,92,383,4,...,2023,7,16,7,7,0.179153,1.673404,1,0.160579,1.849007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620351,620351,2,"{""key4"":""14"",""key5"":""14""}",1688990850527,4,0,41,104,37,4,...,2023,7,10,1,12,0.145397,2.012027,2,0.148067,1.981816
620352,620352,8,unknown,1689490302305,4,0,41,104,37,1,...,2023,7,16,7,6,0.139946,2.075754,1,0.160579,1.849007
620353,620353,41,"{""key1"":""274"",""key2"":""446""}",1688892078584,0,2,41,104,37,4,...,2023,7,9,7,8,0.163471,1.820149,1,0.160579,1.849007
620354,620354,26,"{""key3"":""67804"",""key2"":""650""}",1689652960171,4,0,41,17,366,1,...,2023,7,18,2,4,0.140387,2.070486,0,0.126957,2.241021


In [30]:
def q1(x):
    """
    下四分位数
    """
    return x.quantile(0.25)

def q2(x):
    """
    上四分位数
    """
    return x.quantile(0.75)

In [31]:
aggs = {
    'target': [q1, q2]
}
df_train.groupby('dayofweek').agg(aggs)

target     
              q1   q2
dayofweek            
1            0.0  0.0
2            0.0  0.0
3            0.0  0.0
4            0.0  0.0
5            0.0  0.0
6            0.0  0.0
7            0.0  0.0